# Détection de fraudes de cartes de crédit

## Introduction

L'objectif du projet consiste en l'élaboration d'algorithmes de classification capables de détecter les transactions frauduleuses dans un dataset kaggle de transactions de cartes de crédit. Les données peuvent être trouvées [ici](https://www.kaggle.com/mlg-ulb/creditcardfraud).

Les cas de fraudes ne représentant que 0.173% du nombre total de transactions, l’implémentation des modèles et les métriques de mesures de performance utilisées doivent être adaptées aux données fortement débalancées. Il est également souhaitable que la classification soit sensible, c’est-à-dire que le taux des faux positifs soit minimal, puisque l'institution bancaire ne veut pas déranger inutilement ses clients.


## Plan

* Exploration des données
  * Fléau des données débalancées
* Prétraitement
  * Normalisation
  * Séparation du data
* Régression logistique
* Undersampling aléatoire
  * Sélection de modèle
  * Évaluation


In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Preprocessing
from sklearn.preprocessing import StandardScaler

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Performance metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import confusion_matrix



## Exploration des données

Le dataset contient 284 807 exemples ayant chacun 31 traits caractéristiques numériques dont le temps et le montant de la transaction. Les 28 autres proviennent d'une analyse en composante principale (PCA) effectuée pour anonymiser les données.

In [2]:
df = pd.read_csv("data/creditcard.csv")
print('shape: ', df.shape)
df.head(n=5)

shape:  (284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


Les exemples non-frauduleux sont étiquetés par la classe 0 et les exemples frauduleux par la classe 1. Seulement 492 (0.173%) des exemples sont de la classe 1.

In [4]:
fraud_count = df['Class'].value_counts()[1]
fraud_ratio = np.around(fraud_count/len(df)*100, 3)
print("Nombre de fraudes:", fraud_count)
print("Taux de fraudes:", fraud_ratio, '%')

Nombre de fraudes: 492
Taux de fraudes: 0.173 %



### Fléau des données fortement débalancées

Les données sont extrèmements débalancées. Le classifieur risque de mémoriser que les données sont débalancées et d'assumer que la majorité des exemples sont des cas non-frauduleux (sur-apprentissage) plutôt que de détecter les liens dans les traits caractéristiques permettant réellement de déterminer si une transaction est une fraude.

Les cas de fraudes ne représentant que 0.172% du nombre total de transactions, l’implémentation des modèles et les métriques de mesures de performance utilisées devront être adaptées aux données fortement débalancées. En effet, le taux de classifications (accuracy) n'est pas une mesure de performance naturelle lorsque les données sont débalancées. Pour l'illustrer, considérons un classifieur qui prédit la classe '0' à tous les nouveaux exemples de tests. Un tel classifieur obtiendrait un taux de classifications correctes de 99.827% sans détecter aucune fraude. Un tel classifieur est incapable de généraliser, même si son taux de classifications est élevé.

D'autres métriques sont plus naturelles pour évaluer la performance d'un dataset débalancé. 

Soit $N$ le nombre total d'exemples.

vrais positifs (VP): 

faux positifs (FP): 

vrais négatifs (VN):

faux négaitfs (FN):

Accuracy $:=\frac{VP+VN}{N}$

Precision $:=\frac{VP}{VP+FP}$

Recall $:=\frac{VP}{VP+FN}$

f1-score $:=2\ \frac{\text{precision}\ \times\ \text{recall}}{\text{precision}\ +\ \text{recall}}$

Support:

Le recall nous est particulièrement intéressant comme il s'agit le taux de fraudes détectées.

"Many machine-learning techniques, such as neural networks, make more reliable predictions from being trained with balanced data. Certain analytical methods, however, notably linear regression and logistic regression, do not benefit from a balancing approach." (Wikipedia)

Dotons-nous d'une fonction pour imprimer ces métriques.

In [5]:
def print_metrics(y_true, y_pred, title=''):
    print(title,'\n')
    print('Accuracy:', accuracy_score(y_true, y_pred))
    confusion_m = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_m.ravel()
    
    print('\ntrue positives\tfalse positives\tfalse negatives\ttrue negatives')
    print('%.0f\t\t%.0f\t\t%.0f\t\t%.0f'% (tn, fp, fn, tp))

    metrics = precision_recall_fscore_support(y_true, y_pred, beta=1.0)
    precision = metrics[0]
    recall = metrics[1]
    f1 = metrics[2]
    support = metrics[3]
    print('\nClass\t\t 0\t\t1')
    print('Precision\t', '%.2f\t\t%.2f'% (precision[0], precision[1]))
    print('recall\t\t', '%.2f\t\t%.2f'% (recall[0], recall[1]))
    print('f1\t\t', '%.2f\t\t%.2f'% (f1[0], f1[1]))
    print('support\t\t', '%.2f\t%.2f'% (support[0], support[1]))

## Pré-traitement

### Normalisation
On voudrait normaliser les colonnes Time et Amount comme pour les colonnes V1 à V28. Le but principal de cette normalisation est d'aider les techniques de convergences utilisées pour l'optimisation.

<span style="color:red">Essayer: RobustScaler Scale features using statistics that are robust to outliers.</span>

In [6]:
#Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()

time = df['Time'].values.reshape(-1, 1)
amount = df['Amount'].values.reshape(-1, 1)
norm_time = scaler.fit_transform(time)
norm_amount = scaler.fit_transform(amount)
df.insert(0, 'norm_time', norm_time)
df.insert(0, 'norm_amount',norm_amount)
df.drop(['Time', 'Amount'], axis=1, inplace=True)

df.head()

,norm_amount,norm_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,0.244964,-1.996583,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.342475,-1.996583,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,1.160686,-1.996562,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,0.140534,-1.996562,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,-0.073403,-1.996541,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


### Séparation du data

Avant de sélectionner des modèles, on sépare les données en ensembles d'entraînement et de tests.

Comme les cas de fraudes sont sous-représentés, on voudrait s'assurer que les ensembles de tests et d'entraînements suivent la même distribution. 

In [7]:

inputs, outputs = df.drop('Class', axis=1), df['Class']

# If stratify not None, data is split in a stratified fashion, using this as the class labels.
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.3, random_state=0, stratify=outputs)

# Verify if distributions are the same
print('Ratio de fraudes (Train):', y_train.value_counts()[1]/y_train.shape[0] * 100)
print('Ratio de fraudes (Test):', y_test.value_counts()[1]/y_test.shape[0] * 100)


Ratio de fraudes (Train): 0.17254870488152324
Ratio de fraudes (Test): 0.17321489179921118


## Régression logistique 

Pour commencer, essayons d'implémenter naïvement un classifieur simple comme la régression logistique sur la totalité des données. On utilise le taux de bonnes classifications (accuracy) comme mesure de performance.

Comme discutter précédemment, on s'attend à ce que le modèle mémorise à tord le débalancement dans les données

In [8]:
log_reg = LogisticRegression()

# Tuning penalty and regularization using cross validation
params = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
log_reg_gs= GridSearchCV(log_reg, params, scoring='accuracy', cv=5, verbose=1) # Using accuracy as scoring metric 
log_reg_gs.fit(X_train, y_train)
print('Meilleurs paramètres:', log_reg_gs.best_params_)
print('Meilleur résultat (moyenne des résultats de validation croisée):', log_reg_gs.best_score_)
log_reg_best = log_reg_gs.best_estimator_

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:  3.6min finished


Meilleurs paramètres: {'C': 1, 'penalty': 'l2'}
Meilleur résultat (moyenne des résultats de validation croisée): 0.9992375754900584


In [9]:
# Predict on test set 
predictions = log_reg_best.predict(X_test)

print_metrics(y_test, predictions, 'Régression logistique sur les données débalancées')

Régression logistique sur les données débalancées 

Accuracy: 0.999204147794436

true positives	false positives	false negatives	true negatives
85282		13		55		93

Class		 0		1
Precision	 1.00		0.88
recall		 1.00		0.63
f1		 1.00		0.73
support		 85295.00	148.00


Bien que le taux de bonnes classifications semble élevé, le prédicteur n'a su que détecter 0,63% des fraudes. On voudrait augmenter significativement cette quantité.

## Undersampling aléatoire

L'idée est d'entraîner un modèle sur un ensemble de données balancé tel quel 50% des données sont des fraudes. Modifier ainsi les données d'entraînement introduit un biais qui sert à compenser le risque de surapprentissage dû au fléau des données débalancées.

On sélectionne aléatoirement des données de fraudes et on fait un nouveau dataframe sur lequel on entraîne les modèles. On testera ensuite la performance du modèle le plus performant sur l'ensemble de test original.

<span style="color:red">Try RandomUnderSampler class from the imblearn library. It works by performing k-means clustering on the majority class and removing data points from high-density centroids.</span> 

In [10]:
# Select a random sample of non-fraud exemples
fraud_df = df[df['Class']==1]
non_fraud_df = df[df['Class']==0]
rdm_non_fraud_df = non_fraud_df.sample(n=fraud_df.shape[0])

# New balanced df
under_sample_df = pd.concat([fraud_df, rdm_non_fraud_df])
under_sample_df = under_sample_df.sample(frac=1)

under_sample_df.describe()

,norm_amount,norm_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
count,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,...,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000
mean,0.137474,-0.102755,-2.381699,1.791634,-3.533385,2.291523,-1.636010,-0.706594,-2.733345,0.273733,...,0.223246,0.374509,0.005938,-0.029925,-0.044728,0.007023,0.021970,0.083123,0.027694,0.500000
std,1.438511,1.031471,5.534952,3.779761,6.229667,3.210994,4.213735,1.727820,5.907376,4.877936,...,1.166960,2.805378,1.171909,1.210304,0.566407,0.685655,0.477104,1.002104,0.438657,0.500254
min,-0.353229,-1.996267,-30.552380,-24.426864,-31.103685,-4.559620,-22.105532,-6.406267,-43.557242,-41.044261,...,-4.128186,-22.797604,-8.887017,-19.254328,-2.028024,-4.781606,-1.425404,-7.263482,-2.501568,0.000000
25%,-0.346073,-0.989567,-2.859468,-0.170459,-5.120349,-0.081205,-1.848596,-1.587624,-3.105154,-0.218122,...,-0.184283,-0.174020,-0.524264,-0.245715,-0.373302,-0.358864,-0.283741,-0.069619,-0.062732,0.000000
50%,-0.273468,-0.241542,-0.739646,0.996230,-1.337719,1.360733,-0.488647,-0.616944,-0.635814,0.144982,...,0.037463,0.158980,0.036853,-0.031421,0.009163,0.035358,-0.032957,0.051544,0.035099,0.500000
75%,0.053826,0.904967,1.013464,2.785561,0.347377,4.250632,0.486563,0.126612,0.359971,0.853411,...,0.445619,0.672158,0.560266,0.212723,0.359511,0.395077,0.315862,0.454884,0.214626,1.000000
max,30.177356,1.640120,2.383325,22.057729,3.183450,12.114672,11.095089,7.769639,12.614867,20.007208,...,15.519527,27.202839,8.361985,5.466230,3.641635,2.208209,2.745261,3.052358,1.779364,1.000000


On remarque une augmentation des variances pour les paramètres V1 à V28.

### Sélection de modèle

Peut-être devrait-on choisir une classe de fonction relativement peu expressive (basse capacité) parce que notre dataset n'a que 984 exemples, ce qui signifie qu'il a une grande variabilité. Un algorithme trop expressif risque de mémoriser le bruit et de surapprendre.

Pour augmenter le taux de fraudes détectées, on utilise la métrique recall pour sélectionner le modèle.

Essayons quelques modèles linéaires comme la régression logistique et la machine à vecteurs de support ainsi que KNN (un peu plus expressif). 

In [11]:
# Split undersample (us) data 
inputs, outputs = under_sample_df.drop('Class', axis=1), under_sample_df['Class'] 
X_train_us, X_test_us, y_train_us, y_test_us = train_test_split(inputs, outputs, test_size=0.3, random_state=0)

In [12]:
# Logistic Regression
log_reg_us = LogisticRegression()

# Tuning penalty and regularization using cross validation
params = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
log_reg_us_gs= GridSearchCV(log_reg_us, params, scoring='recall', cv=5, verbose=1)
log_reg_us_gs.fit(X_train_us, y_train_us)
print(log_reg_us_gs.best_params_)
print('Meilleur résultat (moyenne des résultats de validation croisée):', log_reg_us_gs.best_score_)
log_reg_us_best = log_reg_us_gs.best_estimator_

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'C': 0.001, 'penalty': 'l2'}
Meilleur résultat (moyenne des résultats de validation croisée): 0.9675530095759233


[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    7.0s finished
C:\Users\pasca\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


On a probablement augmenté notre capacité à détecter des fraudes avec un recall de 0.95 peut-être au détriment de la précision et de l'Accuracy.

In [13]:
# SVM with gaussian kernel
svm_rbf_us = SVC(kernel='rbf')

# Tuning C
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svm_rbf_us_gs = GridSearchCV(svm_rbf_us, params, scoring='recall', cv=5, verbose=1)
svm_rbf_us_gs.fit(X_train_us, y_train_us)
print(svm_rbf_us_gs.best_params_)
print('Meilleur résultat (moyenne des résultats de validation croisée):', svm_rbf_us_gs.best_score_)
svm_rbf_us_best = svm_rbf_us_gs.best_estimator_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 7 candidates, totalling 35 fits
{'C': 100}
Meilleur résultat (moyenne des résultats de validation croisée): 0.9469036486514077


[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    0.6s finished


In [14]:
# SVM with polynomial kernel
svm_poly_us = SVC(kernel='poly')

# Tuning C and degree
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'degree': [0, 1, 2, 3, 4, 5]}
svm_poly_us_gs = GridSearchCV(svm_poly_us, params, scoring='recall', cv=5, verbose=1)
svm_poly_us_gs.fit(X_train_us, y_train_us)
print(svm_poly_us_gs.best_params_)
print('Meilleur résultat (moyenne des résultats de validation croisée):', svm_poly_us_gs.best_score_)
svm_poly_us_best = svm_poly_us_gs.best_estimator_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 42 candidates, totalling 210 fits
{'C': 10, 'degree': 2}
Meilleur résultat (moyenne des résultats de validation croisée): 0.917405108520326


[Parallel(n_jobs=1)]: Done 210 out of 210 | elapsed:    3.6s finished
C:\Users\pasca\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [15]:
# KNN
KNN_us = KNeighborsClassifier()

# Tuning k
params = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
KNN_us_gs = GridSearchCV(KNN_us, params, scoring='recall', cv=5, verbose=1)
KNN_us_gs.fit(X_train_us, y_train_us)
print(KNN_us_gs.best_params_)
print('Meilleur résultat (moyenne des résultats de validation croisée):', KNN_us_gs.best_score_)
KNN_us_gs_best = KNN_us_gs.best_estimator_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'n_neighbors': 1}
Meilleur résultat (moyenne des résultats de validation croisée): 0.9144560201727342


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.4s finished


In [16]:
# Random Forest
rand_forest_us = RandomForestClassifier()

# Tuning number of trees
params = {'n_estimators':[1, 10, 100, 500]}
rand_forest_us_gs = GridSearchCV(rand_forest_us, params, scoring='recall', cv=5, verbose=1)
rand_forest_us_gs.fit(X_train_us, y_train_us)
print(rand_forest_us_gs.best_params_)
print('Meilleur résultat (moyenne des résultats de validation croisée):', rand_forest_us_gs.best_score_)
rand_forest_us_gs_best = rand_forest_us_gs.best_estimator_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    4.8s finished


{'n_estimators': 500}
Meilleur résultat (moyenne des résultats de validation croisée): 0.9056062029115707


### Évaluation

La régression logistique a les meilleures performances.

In [17]:
# Predict on test set 
predictions_us = log_reg_us_best.predict(X_test)
print('Acc:', log_reg_us_best.score(X_test, y_test), '\n')

print_metrics(y_test, predictions_us, "régression logistique avec sub-sample aléatoire")

Acc: 0.865547792095315 

régression logistique avec sub-sample aléatoire 

Accuracy: 0.865547792095315

true positives	false positives	false negatives	true negatives
73815		11480		8		140

Class		 0		1
Precision	 1.00		0.01
recall		 0.87		0.95
f1		 0.93		0.02
support		 85295.00	148.00


## Références
[Addressing the Curse of Imbalanced Training Sets: One-Sided Selection](https://sci2s.ugr.es/keel/pdf/algorithm/congreso/kubat97addressing.pdf)

[Credit Fraud Detector (notebook kaggle)](https://www.kaggle.com/kernels/scriptcontent/16695845/download)

[Toward data science article](https://towardsdatascience.com/detecting-financial-fraud-using-machine-learning-three-ways-of-winning-the-war-against-imbalanced-a03f8815cce9)

[Evaluation Metrics, ROC-Curves and imbalanced datasets](http://www.davidsbatista.net/blog/2018/08/19/NLP_Metrics/)

[Subsampling and Oversampling Wikipedia](https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis#Undersampling_techniques_for_classification_problems)